# EV Charging Data Cleaning & Preparation

1. This notebook standardizes timestamps, validates durations, removes invalid and extreme outliers, and saves a clean dataset for modeling.
2. Merges datasets from multiple sources to create a comprehensive view of EV charging sessions.
   1. Dataset 1: EV Charging Sessions_hourly (data/trondheim/Dataset_2_Hourly_EV_per_user.csv)
   2. Dataset 2: Trondheim weather (data/trondheim/Norway_Trondheim_ExactLoc_Weather.csv)
   3. Dataset 3: EV Charging Reports (data/trondheim/Dataset 1_EV charging reports.csv)

The Logic: A user plugging in at 5 PM might stay for 2 hours (shopping) or 12 hours (overnight). The Time is the same. The difference is the Temperature (cold batteries charge slower) and the User Habit.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta

pd.set_option('display.max_columns', None)

RAW_PATH = '../../data/trondheim/Dataset 1_EV charging reports.csv'
OUT_DIR = 'data'
OUT_FILE = os.path.join(OUT_DIR, 'ev_sessions_clean.csv')

print('Configured paths:')
print('  RAW_PATH:', RAW_PATH)
print('  OUT_FILE:', OUT_FILE)

Configured paths:
  RAW_PATH: ../../data/trondheim/Dataset 1_EV charging reports.csv
  OUT_FILE: data/ev_sessions_clean.csv


Load Dataset 1_EV charging reports.csv.
Load Norway_Trondheim_ExactLoc_Weather.csv.
Merge: You must merge them on the Start_plugin timestamp.
Code Hint: Round the session start time to the nearest hour, then merge with the weather dataframe.

## Quick sanity checks (run first)

These cells load the cleaned dataset, report duration statistics and candidate target class balances, and run a short merge diagnostic with the weather file.

Paste and run these cells before further merging or modeling — they will flag any class-balance or merge-key issues immediately.

In [ ]:
# Sanity check: load cleaned dataset and inspect duration / candidate targets
import os
import pandas as pd

clean_path = os.path.join('data','ev_sessions_clean.csv')
print('Checking cleaned dataset at:', clean_path)
if not os.path.exists(clean_path):
    print('File not found:', clean_path)
else:
    # parse common datetime columns if present
    try:
        df = pd.read_csv(clean_path, parse_dates=['Start_plugin_dt','End_plugout_dt'], dayfirst=True)
    except Exception:
        df = pd.read_csv(clean_path)
    print('shape:', df.shape)
    try:
        from IPython.display import display
        display(df.head())
    except Exception:
        print(df.head().to_string())

    # Ensure we have a Duration_hours column (some notebooks use Duration_hours_calc)
    if 'Duration_hours' not in df.columns:
        if 'Duration_hours_calc' in df.columns:
            df['Duration_hours'] = df['Duration_hours_calc']
            print('Using Duration_hours_calc -> Duration_hours')
        else:
            print('WARNING: no Duration_hours column found. Check cleaning notebook outputs.')

    if 'Duration_hours' in df.columns:
        print('\nDuration_hours summary (selected percentiles):')
        print(df['Duration_hours'].describe(percentiles=[0.5,0.75,0.9,0.95,0.99]))

        # Candidate thresholds: 12h, 24h, 48h
        for t in [12, 24, 48]:
            prop = (df['Duration_hours'] < t).mean()
            cnt = int((df['Duration_hours'] < t).sum())
            print(f"<{t}h : {prop:.4f} ({cnt} samples)" )

        # Create a 24h binary target and show distribution (normalize)
        df['is_short_24'] = (df['Duration_hours'] < 24).astype(int)
        print('\nTarget (is_short_24) distribution (normalize):')
        print(df['is_short_24'].value_counts(normalize=True))
        print('Absolute counts:')
        print(df['is_short_24'].value_counts())

        short_prop = df['is_short_24'].mean()
        if short_prop > 0.90:
            print('WARNING: short class proportion > 90% — accuracy will be misleading. Use precision/recall on LONG class.')

        p90 = df['Duration_hours'].quantile(0.90)
        print('90th percentile duration:', p90)

        # Save small sample for downstream merge diagnostics
        df_sample = df[['Start_plugin_dt','Duration_hours']].copy() if 'Start_plugin_dt' in df.columns else df.head(100).copy()
        # Keep df in notebook namespace for following cells
        globals()['df'] = df
        globals()['df_sample'] = df_sample

In [ ]:
# Merge diagnostics: attempt to locate weather datetime column and report match rate
import os
import pandas as pd

weather_path = os.path.join('..','..','data','trondheim','Norway_Trondheim_ExactLoc_Weather.csv')
print('Checking weather file at:', weather_path)
if not os.path.exists(weather_path):
    print('Weather file not found at expected path. Adjust path and re-run.')
else:
    # load with minimal parsing, we'll try to find datetime column(s)
    weather = pd.read_csv(weather_path)
    print('Weather columns:', list(weather.columns)[:30])
    # heuristics to find datetime-like column
    dt_candidates = [c for c in weather.columns if any(k in c.lower() for k in ['date','time','dt','timestamp','hour'])]
    if len(dt_candidates) == 0:
        # try parsing first object column
        for c in weather.columns:
            if weather[c].dtype == object:
                parsed = pd.to_datetime(weather[c], errors='coerce', dayfirst=True)
                if parsed.notna().sum() > 0:
                    weather['weather_dt_candidate'] = parsed
                    dt_candidates = ['weather_dt_candidate']
                    break
    if len(dt_candidates) == 0:
        print('No datetime-like column auto-detected in weather file — open it to choose the correct column for merging.')
    else:
        wc = dt_candidates[0]
        print('Using weather datetime column:', wc)
        weather['weather_dt'] = pd.to_datetime(weather[wc], errors='coerce', dayfirst=True)
        weather['weather_hour'] = weather['weather_dt'].dt.round('H')
        # Ensure df and Start_plugin_dt exist from previous cell
        if 'df' not in globals():
            print('df not available in namespace — run the previous sanity-check cell first')
        else:
            if 'Start_plugin_dt' not in df.columns:
                print('df missing Start_plugin_dt — ensure cleaning notebook produced this column')
            else:
                df['start_hour'] = df['Start_plugin_dt'].dt.round('H')
                merged = df.merge(weather[['weather_hour']], left_on='start_hour', right_on='weather_hour', how='left', indicator=True)
                print('Weather merge indicator counts:')
                print(merged['_merge'].value_counts())
                matched = merged['_merge'].eq('both').mean()
                print(f'Percent matched to weather: {matched:.2%}')
                # keep merged sample for inspection
                globals()['merged_sample'] = merged.head(20)
                try:
                    from IPython.display import display
                    display(merged.head())
                except Exception:
                    print(merged.head().to_string())

### Next steps

- If the 24h target is extremely imbalanced (e.g. >90% short), consider alternative targets (12h, quantile-based) or treat as rare-event problem.
- If weather merge match rate is low, re-check rounding strategy (floor vs round) and timezone handling.
- Once these checks pass, apply the existing feature-engineering cells from the cleaning notebook to this merged dataset.

## Raw-dataset scaffold: load & diagnostics

This cell loads the three raw files (reports, hourly per-user, weather), prints shapes and sample rows, auto-detects datetime-like columns and common join keys, and computes simple overlap diagnostics. Run this before cleaning so we can confirm the raw inputs and choose merge keys.


In [ ]:
# Scaffold diagnostics: load raw files and inspect
import os
import pandas as pd

base = os.path.join('..','..','data','trondheim')
paths = {
    'reports': os.path.join(base, 'Dataset 1_EV charging reports.csv'),
    'hourly': os.path.join(base, 'Dataset_2_Hourly_EV_per_user.csv'),
    'weather': os.path.join(base, 'Norway_Trondheim_ExactLoc_Weather.csv')
}

loaded = {}
for name, p in paths.items():
    print(f"\n-> {name}: {p}")
    if not os.path.exists(p):
        print('   MISSING')
        loaded[name] = None
        continue
    try:
        # try light parsing (don't force too many datetimes yet)
        df_raw = pd.read_csv(p, low_memory=False)
    except Exception as e:
        print('   Failed to read:', e)
        loaded[name] = None
        continue
    loaded[name] = df_raw
    print('   shape:', df_raw.shape)
    print('   columns (first 20):', list(df_raw.columns)[:20])
    display(df_raw.head(3))

# helper to auto-detect datetime-like columns
def detect_datetime_cols(df):
    candidates = []
    for c in df.columns:
        lc = c.lower()
        if any(k in lc for k in ['date','time','start','end','dt','timestamp','hour']):
            candidates.append(c)
    # try parsing a few object columns if none found
    if len(candidates) == 0:
        for c in df.select_dtypes(include=['object']).columns[:10]:
            parsed = pd.to_datetime(df[c], errors='coerce', dayfirst=True)
            if parsed.notna().sum() > 0.01 * len(df):
                candidates.append(c)
    return candidates

# detect datetime cols
for name, df_raw in loaded.items():
    if df_raw is None:
        continue
    print(f"\n{name} datetime candidates:")
    dtc = detect_datetime_cols(df_raw)
    print(dtc)

# detect possible key columns (station/user)
key_names = ['Garage_ID','GarageID','StationID','LocationID','Shared_ID','SharedId','SharedID','UserID','user_id']
for name, df_raw in loaded.items():
    if df_raw is None:
        continue
    found = [k for k in key_names if k in df_raw.columns]
    print(f"\n{name} found key columns: {found}")
    for k in found:
        print(f"  unique {k}: {df_raw[k].nunique()} values, top 5:\n", df_raw[k].value_counts().head(5))

# basic hourly distribution for reports if Start_plugin available
if loaded.get('reports') is not None:
    rep = loaded['reports'].copy()
    # try common start col names
    start_cols = [c for c in rep.columns if 'start' in c.lower() or 'plugin' in c.lower()]
    print('\nreports candidate start cols:', start_cols[:5])
    if len(start_cols) > 0:
        col = start_cols[0]
        rep['_parsed_start'] = pd.to_datetime(rep[col], dayfirst=True, errors='coerce')
        print('Parsed start nulls:', rep['_parsed_start'].isna().sum())
        if rep['_parsed_start'].notna().any():
            rep['start_hour'] = rep['_parsed_start'].dt.round('H')
            print('Sessions per start_hour (sample):')
            print(rep['start_hour'].dt.floor('D').value_counts().sort_index().head())

# quick merge overlap example: reports <> weather on rounded hour
if loaded.get('reports') is not None and loaded.get('weather') is not None:
    rpt = loaded['reports'].copy()
    wth = loaded['weather'].copy()
    # detect and parse start in reports
    rpt_start_candidates = detect_datetime_cols(rpt)
    wth_start_candidates = detect_datetime_cols(wth)
    print('\nUsing rpt_start_candidates:', rpt_start_candidates)
    print('Using wth_start_candidates:', wth_start_candidates)
    if len(rpt_start_candidates) > 0 and len(wth_start_candidates) > 0:
        rpt['_s'] = pd.to_datetime(rpt[rpt_start_candidates[0]], dayfirst=True, errors='coerce')
        wth['_w'] = pd.to_datetime(wth[wth_start_candidates[0]], dayfirst=True, errors='coerce')
        rpt['start_hour'] = rpt['_s'].dt.round('H')
        wth['weather_hour'] = wth['_w'].dt.round('H')
        merged_example = rpt.merge(wth[['weather_hour']].drop_duplicates(), left_on='start_hour', right_on='weather_hour', how='left', indicator=True)
        print('Weather merge indicator counts (example):')
        print(merged_example['_merge'].value_counts())
        print('Percent matched:', merged_example['_merge'].eq('both').mean())
    else:
        print('Cannot auto-detect suitable datetime columns for merge example — inspect the candidate lists above.')
